# Checking motif overlap
We want to see, for motifs that appear enriched in CND, if they are also present in ATL. We used `annotatePeaks.pl` from HOMER for the heavy lifting here; we now just need to read in the inputs and summarize them.

In [124]:
from os import listdir
from os.path import splitext, isfile
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from scipy.stats import fisher_exact, false_discovery_control
import matplotlib.pyplot as plt
from itertools import product
import json

## Reading in the data
Data are stored as tab-delimited files. However, most of the rows are empty of useful information; each file contains every promoter in the entire ATL genome, with only a few rows containing something in the column that shows motif matches. Unfortunately that column does not have a consisten name; however, it is always the last one. Additionally, the RV motif output files should be semantically identical to the forward ones, with the possible exception of the sequence being flipped in the macthed motif. We will conserve both for the moment in order to check them later.

In [2]:
data_dir = '../data/outputs/annotated_motifs/CND_searching_ATL_timey_background_ShD_16Apr2025'
motif_matches = defaultdict(dict)
for f in tqdm(listdir(data_dir)):
    if isfile(f'{data_dir}/{f}'):
        module = '_'.join(splitext(f)[0].split('_')[:2])
        motif = splitext(f)[0].split('_')[2]
        df = pd.read_csv(f'{data_dir}/{f}', sep='\t', header=0)
        motif_matches[module][motif] = df

 15%|████████████▏                                                                   | 203/1333 [02:01<11:24,  1.65it/s]/tmp/ipykernel_3793586/68569872.py:7: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{data_dir}/{f}', sep='\t', header=0)
 15%|████████████▏                                                                   | 204/1333 [02:02<11:19,  1.66it/s]/tmp/ipykernel_3793586/68569872.py:7: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{data_dir}/{f}', sep='\t', header=0)
100%|███████████████████████████████████████████████████████████████████████████████| 1333/1333 [14:08<00:00,  1.57it/s]


### Check that the forward and reverse identifications are the same
There are two main things to check: first, that the same rows are present in the dataframe, and second, that the identified sequences are either identical or just reverse complements. But first, did every motif have a reverse complement checked?

In [41]:
no_rev = 0
mismatch = 0
for module, motifs in motif_matches.items():
    for motif, df in motifs.items():
        if 'RV' not in motif:
            rev_motif = motif + 'RV'
            if rev_motif not in motifs.keys():
                no_rev += 1
            else:
                # Check if the contents are the same
                try:
                    # Rather than checking entire df, just check if the gene names with
                    # non-NaN in the column of interest are the same
                    gset1 = set(motifs[motif][~motifs[motif].iloc[:, -1].isna()].iloc[:, 0].tolist())
                    gset2 = set(motifs[rev_motif][~motifs[rev_motif].iloc[:, -1].isna()].iloc[:, 0].tolist())
                    assert gset1 == gset2
                except AssertionError as e:
                    mismatch += 1

In [42]:
no_rev, mismatch

(0, 0)

The reverses and the forward do in fact identify the same genes! This means we can remove any items that have RV in the motif name:

In [43]:
motif_matches_forward_only = {
    mod: {mot: v for mot, v in motifs.items() if 'RV' not in mot}
    for mod, motifs in motif_matches.items()
}

### Reading in the module data
We also need to check whether or not the matches are within the same module, so we need to read in the module memberships. We need to replace underscores with periods here to match the values in the gene ID column of our motif matching results:

In [55]:
module_base_path = '/home/farre/Ann/RNAseq/HOMER_analysis/queries/shd_pca_modulesapril2024'
module_members = {'ATL': {}, 'CND': {}}
for module in listdir(module_base_path):
    if 'ATL_ShD_Leaf_vs_CND_ShD_Leaf_' in module and '.txt' in module:
        module_basename = splitext(module)[0][len('ATL_ShD_Leaf_vs_CND_ShD_Leaf_'):-len('_in_XXX')]
        species = splitext(module)[0][-3:]
        with open(f'{module_base_path}/{module}') as f:
            module_genes = [l.strip() for l in f.readlines()]
        module_genes = [s[:s.rfind('_')] + '.' + s[s.rfind('_')+1:] for s in module_genes]
        module_members[species][module_basename] = module_genes

For our Fisher's Exact test we'll also need a complete list of the genes in all modules for ATL.

In [57]:
all_module_genes_ATL = []
for mod, members in module_members['ATL'].items():
    all_module_genes_ATL += members

## Motif matching
Now we go through each module's motif matches and see how many genes within that module have motif matches. We searched ATL with the motifs we identified in CND, so we will need the ATL gene ID's.

In [58]:
fisher_cats = defaultdict(dict)
for module, results in motif_matches_forward_only.items():
    print(f'On module {module} of {len(motif_matches_forward_only)}')
    for motif, matches in tqdm(results.items(), leave=False):
        in_module = matches[matches.iloc[:, 0].isin(module_members['ATL'][module])]
        in_mod_w_motif = in_module[~in_module.iloc[:, -1].isna()]
        in_mod_wo_motif = in_module[in_module.iloc[:, -1].isna()]
        out_mod = [g for g in all_module_genes_ATL if g not in in_module.iloc[:, 0].tolist()]
        out_mod_all = matches[matches.iloc[:, 0].isin(out_mod)]
        out_mod_w_motif = out_mod_all[~out_mod_all.iloc[:, -1].isna()]
        out_mod_wo_motif = out_mod_all[out_mod_all.iloc[:, -1].isna()]
        fisher_cats[module][motif] = {
            'in_mod_with_motif': in_mod_w_motif,
            'in_mod_without_motif': in_mod_wo_motif,
            'out_mod_with_motif': out_mod_w_motif,
            'out_mod_without_motif': out_mod_wo_motif
        }

On module module_0 of 18


On module module_1 of 18


On module module_2 of 18


On module module_25 of 18


On module module_26 of 18


...

On module module_4 of 18


On module module_42 of 18


On module module_5 of 18


On module module_6 of 18


On module module_77 of 18


## Statistical testing
We now want to perform a Fisher's exact test on each motif/module combination to see if it's enriched in ATL in addition to CND (which we already know from the HOMER analysis). Fisher's will also test positive if the motif appears many times in one group and none in the other, so we will have to visually check the contingecy tables for those that are statistically significant.

First, get the contingency tables:

In [59]:
fisher_numbers = {
    mod: {
        motif: [[len(m['in_mod_with_motif']), len(m['in_mod_without_motif'])], [len(m['out_mod_with_motif']), len(m['out_mod_without_motif'])]]
         for motif, m in motifs.items()    
    }
    
    for mod, motifs in fisher_cats.items()
}

Perform the test:

In [77]:
fisher_results = {
    mod: {
        motif: fisher_exact(table, alternative='greater') for motif, table in motifs.items()
    }
    for mod, motifs in fisher_numbers.items()
}

We need to apply a multiple testing correction, which we'll do with `scipy.stats.false_discovery_control` using the `Benjaminini-Yekutieli` correction. First, we need to flatten the results so we can provide a 1D array of p-values to be corrected.

In [81]:
to_correct = []
to_correct_names = []
for module, motifs in fisher_results.items():
    for motif, res in motifs.items():
        to_correct.append(res[1])
        to_correct_names.append(module + '_' + motif)

Now apply the correction:

In [82]:
corrected_ps = false_discovery_control(to_correct, method='by')

How much did this impact the number of motifs per module that come back significant?

In [86]:
len([p for p in to_correct if p < 0.05]), len([p for p in corrected_ps if p < 0.05])

(560, 437)

Filter out the non-significant results as we combine the names and p-values again:

In [92]:
corrected_p_results = {cn: cp for cn, cp in zip(to_correct_names, corrected_ps) if cp < 0.05}

## Cross-module comparisons
While we've been able to somewhat narrow down the number of motifs of interest by checking if they're also enriched in the same module in ATL as in CND where they were identified, we still have a prohibitively large number of motifs. Another way we can narrow these down is to consider that we want motifs that appear in multiple modules. In particular, we want to identify motifs that appear in multiple "shifted" modules, where shifting means that ATL is late compared to CND, which is our phenotype of interest. To do this, we'll take a similar approach as above, but instead of checking the motifs within their original modules, we will also check them across other modules. We will only consider those that tested significant in the previous section.

To make this easier, flatten all the data by making composite names containing both module and motif numbers:

In [113]:
flattened_matched_motifs = {f'{module}_{motif}': matches for module, motifs in motif_matches_forward_only.items() for motif, matches in motifs.items() if f'{module}_{motif}' in corrected_p_results.keys()}
len(flattened_matched_motifs)

437

We now need every pairwise comparison of module and motif, and to create the Fisher's contingency table for each of these comparisons.

In [115]:
module_names = list(motif_matches_forward_only.keys())
motif_names = list(flattened_matched_motifs.keys())

I'm going to streamline the process of creating the contingency table and doing the testing so we don't need to loop over all the combinations multiple times:

In [121]:
cross_fisher_results = {}
for module, motif in tqdm(product(module_names, motif_names)):
    in_module = matches[matches.iloc[:, 0].isin(module_members['ATL'][module])]
    in_mod_w_motif = len(in_module[~in_module.iloc[:, -1].isna()])
    in_mod_wo_motif = len(in_module[in_module.iloc[:, -1].isna()])
    out_mod = [g for g in all_module_genes_ATL if g not in in_module.iloc[:, 0].tolist()]
    out_mod_all = matches[matches.iloc[:, 0].isin(out_mod)]
    out_mod_w_motif = len(out_mod_all[~out_mod_all.iloc[:, -1].isna()])
    out_mod_wo_motif = len(out_mod_all[out_mod_all.iloc[:, -1].isna()])
    contingency_table = [[in_mod_w_motif, in_mod_wo_motif], [out_mod_w_motif, out_mod_wo_motif]]
    fisher_res = fisher_exact(contingency_table, alternative='greater')
    cross_fisher_results[f'{module}__{motif}'] = fisher_res

7866it [1:39:36,  1.32it/s]


Save the results so we don't need to run it again:

In [125]:
with open('../data/statistical_testing/29Apr2025_cross_module_fisher.json', 'w') as f:
    json.dump(cross_fisher_results, f)

Perform multiple testing correction:

In [130]:
cross_ps = [res[1] for res in cross_fisher_results.values()]
corrected_cross_ps = false_discovery_control(cross_ps, method='by')

Impact of the correction:

In [129]:
len([p for p in cross_ps if p < 0.05]), len([p for p in corrected_cross_ps if p < 0.05])

(874, 0)